# Trabalho 2 - Tópicos em IA
#### Aluno: Matheus Bernard Mota
#### RGA: 2022.1904.008-6

---



## Eficácia de Diferentes Técnicas de Pooling na Detecção de Fake News

Neste notebook, utilizaremos o modelo BERT pré-treinado em pt-br **BERTimbau** com uma camada linear para classificação.
Para o Fine-tuning, utilizaremos o dataset de *fake-news* Fake.br corpus. A ideia é, a partir do modelo pré treinado, especializá-lo em: a partir de uma sequência (notícia), corretamente predizer se ela é verdadeira **(true)** ou falsa, **(fake)**.

O código fonte foi modularizado para melhor organização, compreensão e manutenibilidade e um notebook explicativo foi posto para direcionar a execução do procedimento e análise. 

---

## Procedimentos Iniciais
1. Anexar a pasta .src com este notebook, para que ele tenha acesso aos seus módulos
2. Realizar os imports iniciais
3. Mostra o device aceito pelo torch (#visualizações_adicionais)

In [1]:
#permite importar src

import os, sys
# import sys

root_relative_path = '../'
root = os.path.abspath(root_relative_path)

sys.path.append(root)

In [ ]:
#importa tudo aquilo que importa
import torch
import random
import numpy as np
import evaluate

from src.dataset_loader import DatasetLoader
from src.custom_bertimbau_classifier import CustomBertimbauClassifier
# from src.baseline_bertimbau_classifier import BaselineBertimbauClassifier
from transformers import (
    BertForSequenceClassification, 
    EarlyStoppingCallback  #adicionado após ver que, na validação com 2 épocas, após o step 450, o modelo piorou a loss
)
from src.fine_tuner import FineTuner
import src.config as cfg


c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#physics are relative, but this code gonna be DETERMINISTIC BABY!
random.seed(cfg.RANDOM_SEED) 
np.random.seed(cfg.RANDOM_SEED) 
torch.manual_seed(cfg.RANDOM_SEED)

torch.backends.cudnn.deterministic = True

In [4]:
print(cfg.DEVICE)

cpu


## Carregando o Dataset de Notícias

O dataset contém 7200 notícias, com labels num ratio de ~50%

#### Under the hood

O que este loader está fazendo?
1. Acessando o arquivo **pre-processed.csv** de um clone do repositório do Fake.br. (está no .gitignore, em caso de configuração local, este detalhe deve ser levado em conta)
2. Carregando um tokenizer, proveniente do próprio bertimbau, para traduzir o texto do corpus em tokens processados 
3. Separando as colunas dos textos e suas respectivas "labels", ou seja, classificação da veracidade da notícia
4. Separando, destas colunas, uma porcentagem para treino, validação(ajuste manual de parâmetros) e teste
5. transformando estes dados em tokens e preparando um dataset anexavel ao Trainer da biblioteca Transformers

In [5]:
path = os.path.join(root, cfg.PATH_TO_DATASET)
ds_loader = DatasetLoader(
    path=path,
    model_name=cfg.BERTIMBAU,
    max_len=cfg.SEQ_LEN
)

#conjunto para teste, validação e treino
train_dataset, val_dataset, test_dataset = (ds_loader
                                            .load_dataset(seed=cfg.RANDOM_SEED)
                                            .get_datasets())



In [6]:
#apenas para checar se tá tudo ok
print(train_dataset.encodings)
print(train_dataset[0])
print(repr(train_dataset))

{'input_ids': tensor([[  101,  6969,   304,  ...,     0,     0,     0],
        [  101,  2779, 22278,  ...,   877,   151,   102],
        [  101,   599,   266,  ..., 22281,  6185,   102],
        ...,
        [  101,   146, 12764,  ...,  3573,  1640,   102],
        [  101,  6171, 20027,  ...,     0,     0,     0],
        [  101,  4485,  1797,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([  101,  6969,   304, 11489,   562,  1315,  1418,  2354, 22279, 20808,
         6248,  7635, 22278, 12456,  1502,  1004, 12456, 

## Carregando modelos BERTimbau

#### **baseline_model**
Versão do modelo onde o pooling é feito pela passagem do token **[CLS]**, presente no início de cada sentença, de modo a capturar o contexto geral de todo o documento. Este vetor é utilizado como entrada na camada de classificação.

#### **alternative_model**
Similarmente ao **baseline_model**, este modelo também aproveita do contexto capturado pelo token **[CLS]**, mas ele é concatenado com um vetor que calcula a média dos valores de todos os tokens presentes em cada sentença. Ou seja, se tenho X senteças de tamanho n  (os tamanhos variam, mas isso é regularizado com o token [PAD], que está configurado para não ser levado em consideração nas manipulações matriciais), cada i-ésimo token dos n tokens de cada sentença é somado X vezes, e uma média é retirada, divindindo o valor pelos numero de tokens significativos somados. Essa é outra forma de realizar o pooling, e garante um vetor de tamanho igual a de [CLS]. Estes dois vetores são então concatenados (dobrando o tamanho das features) e levado para a camada de classificação.

$$\text{mean\_pooling}(\mathbf{H}, \mathbf{M}) = \frac{\sum_{i=1}^{n} \mathbf{h}_i \cdot m_i}{\sum_{i=1}^{n} m_i}$$

onde:
- $\mathbf{H}$ é a matriz de embedding
- $\mathbf{M}$ é a matriz de atenção, responsável por eliminar os tokens [PAD] da conta 

$$\text{concat\_pooling}(\mathbf{H}, \mathbf{M}) = [\mathbf{h}_{[CLS]}; \text{mean\_pooling}(\mathbf{H}, \mathbf{M})]$$



In [7]:
model_kwargs = ds_loader.get_labels_mapping()

alternative_model = CustomBertimbauClassifier(cfg.BERTIMBAU, **model_kwargs).to(cfg.DEVICE)
# baseline_model = BaselineBertimbauClassifier(cfg.BERTIMBAU, **model_kwargs).to(cfg.DEVICE)
baseline_model = (BertForSequenceClassification
                    .from_pretrained(
                        cfg.BERTIMBAU,
                        **model_kwargs
                    )
                    .to(cfg.DEVICE))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:

print("Configuraç~ao dos Modelos:")
print(baseline_model)
print("___________________________________________")
print(alternative_model)

Configuraç~ao dos Modelos:
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm

## Fine tunando os modelos

In [ ]:
#ADICIONANDO AS MÉTRICAS SUGERIDAS NA AVALIAÇÃO
#------------------------------------------------------
acc_metric = evaluate.load("accuracy")
prec_metric = evaluate.load("precision")
rec_metric = evaluate.load("recall")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    
    return {
        "accuracy": acc_metric.compute(predictions=preds, references=labels)["accuracy"],
        "precision": prec_metric.compute(predictions=preds, references=labels, average="macro")["precision"],
        "recall": rec_metric.compute(predictions=preds, references=labels, average="macro")["recall"],
        "f1": f1_metric.compute(predictions=preds, references=labels, average="macro")["f1"]
    }

#EARLY STOPPING CALLBACK TECHNIQUE, 
# to prevent loss increasing (occured as said right above)

early_stopping_callback = EarlyStoppingCallback( early_stopping_patience=2 ) 

#FINE TUNING PIPELINE
#trouxe o pipeline para fora da classe, pois é uma peça importante que vale a pena estar a mostra
#------------------------------------------------------
def fine_tuning_pipeline(fine_tuner: FineTuner, model: torch.nn.Module, output_dir: str):
    global root, train_dataset, val_dataset, compute_metrics, cfg, early_stopping_callback
    return (
        fine_tuner
        .set_model(model)
        .set_compute_metrics(compute_metrics)
        .set_trainer_optimizer_params(lr_bert=cfg.LEARNING_RATE)
        .set_training_arguments(
            metric_for_best_model="f1",
            greater_is_better=True, #adicionado para o early stopping
            load_best_model_at_end=True, #adicionado para o early stopping
            output_dir=os.path.join(root, output_dir),
            evaluation_strategy="epoch",
            save_strategy="epoch",
            learning_rate=cfg.LEARNING_RATE,
            weight_decay=0.01,
            per_device_train_batch_size=cfg.BATCH_SIZE,
            per_device_eval_batch_size=cfg.BATCH_SIZE,
            num_train_epochs=cfg.NUM_EPOCHS,
            fp16=torch.cuda.is_available(),
            logging_steps=90,
            report_to="none",
            seed=cfg.RANDOM_SEED,
        )
        .set_trainer(
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            callbacks=[early_stopping_callback],
        )
        .train())

In [ ]:

fine_tuner = FineTuner(
    tokenizer=ds_loader.get_tokenizer())

tuned_baseline_res = fine_tuning_pipeline(
    output_dir="bertimbau-baseline-cls-ptbr",
    fine_tuner=fine_tuner,
    model=baseline_model
)

In [11]:
tuned_baseline_res

NameError: name 'tuned_baseline_res' is not defined

In [ ]:

#bem que eu poderia criar um método de reset
fine_tuner = FineTuner(
    tokenizer=ds_loader.get_tokenizer())

tuned_alternative_res =  fine_tuning_pipeline(
    fine_tuner=fine_tuner,
    model=alternative_model,
    output_dir="bertimbau-alternativo-cls-ptbr",
)

[Aviso] parâmetros ignorados nesta versão: ['evaluation_strategy']


Step,Training Loss
90,0.588100
180,0.376800
270,0.250800
360,0.203500
450,0.099600
540,0.109800
630,0.108900


c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\mathe\OneDrive\Área de Trabalho\fake-news-detector\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [13]:
tuned_alternative_res

{'eval_loss': 0.21633681654930115,
 'eval_accuracy': 0.9481481481481482,
 'eval_precision': 0.9483018867924529,
 'eval_recall': 0.9481481481481482,
 'eval_f1': 0.9481437023064392,
 'eval_runtime': 190.118,
 'eval_samples_per_second': 5.681,
 'eval_steps_per_second': 0.358,
 'epoch': 2.0}